In [1]:
import sys
import os
sys.path.append(os.path.abspath("../.."))
import numpy as np
import yfinance as yf
import plotly.express as px
from tinyshift.series import *
from tinyshift.stats import trailing_window, mad
from numpy.random import standard_normal

In [2]:
ticker = 'PETR4.SA'
period = '2y'
data = yf.download(ticker, period=period, interval="1d", auto_adjust=True)
t = np.linspace(0, 10, 1000)
pure_sine = np.sin(2 * np.pi * 1 * t)  # Perfectly forecastable

[*********************100%***********************]  1 of 1 completed


In [3]:
r = standardize_returns(data["Close"]["PETR4.SA"], log=False)

# RSI

In [4]:
vol_rsi = relative_strength_index(data["Close"]["PETR4.SA"], rolling_window=14)

In [5]:
px.line(vol_rsi, title=f"{ticker} Closing Prices").show(fig_type="png")

# Entropy Volatility

In [6]:
vol_ent = trailing_window(r, rolling_window=60, func=sample_entropy, m=1)

In [7]:
px.line(vol_ent, title=f"{ticker} Closing Prices").show(fig_type="png")

In [8]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    y=data["Close"]["PETR4.SA"],
    x=data.index,
    name="Preço PETR4.SA",
    yaxis="y1"
))

fig.add_trace(go.Scatter(
    x=data.index,
    y=vol_ent,
    name="Volatility Entropy",
    yaxis="y2"
))

fig.add_trace(go.Scatter(
    x=data.index,
    y=vol_rsi,
    name="Volatility RSI",
    yaxis="y3"
))


fig.update_layout(
    yaxis3=dict(
        title="Volatility RSI",
        anchor="free",
        overlaying="y",
        side="right",
        position=1.0,
        showgrid=False
    )
)

fig.update_layout(
    title="Preço PETR4.SA e Volatility Entropy",
    xaxis_title="Data",
    yaxis=dict(title="Preço PETR4.SA"),
    yaxis2=dict(title="Volatility Entropy", overlaying="y", side="right"),
    legend=dict(x=0.01, y=0.99)
)

fig.show(fig_type="png")

# Bollinger's Band

In [9]:
np.sum(bollinger_bands(data["Close"]["PETR4.SA"]))

np.int64(55)

In [10]:
np.sum(bollinger_bands(data["Close"]["PETR4.SA"], center=np.median, spread=mad))

np.int64(173)

# Hampel Filter

In [11]:
hf = hampel_filter(data["Close"]["PETR4.SA"], rolling_window=7)

In [12]:
data["Close"]["PETR4.SA"][hf]

Date
2024-05-22    30.161890
2024-07-04    31.138037
2024-10-09    31.938404
2025-02-07    33.884270
Name: PETR4.SA, dtype: float64

# ADI-CV Framework

In [13]:
adi_cv(r)

(np.float64(1.0), np.float64(7.922816251426432e+34))

# Sampen

In [14]:
sample_entropy(r, m=3)

np.float64(1.6942611047431773)

In [15]:
sample_entropy(pure_sine, m=2)

np.float64(0.16791509477225003)

# Permutation Entropy

In [16]:
permutation_entropy(r, m=3, delay=1, normalize=False)

np.float64(2.584119005504062)

In [17]:
permutation_entropy(r, m=3, delay=1, normalize=True)

np.float64(0.9996736915073782)

In [18]:
permutation_entropy(data["Close"]["PETR4.SA"], m=3, delay=1, normalize=False)

np.float64(2.487321011069855)

# Stability Index

In [19]:
stability_index(r)

np.float64(0.1357412653446648)

In [20]:
stability_index(pure_sine)

np.float64(0.8656234267494547)

# Theorical Limit

In [21]:
theoretical_limit(data["Close"]["PETR4.SA"], m=4, delay=1)

np.float64(0.07754109228495298)

In [22]:
theoretical_limit(pure_sine, m=4, delay=1)

np.float64(0.7114310780157527)

# Foreca

In [23]:
foreca(r)

0.0722612339855565

In [24]:
trailing_window(r, rolling_window=60, func=foreca)

array([0.13470088, 0.13470088, 0.13470088, 0.13470088, 0.13470088,
       0.13470088, 0.13470088, 0.13470088, 0.13470088, 0.13470088,
       0.13470088, 0.13470088, 0.13470088, 0.13470088, 0.13470088,
       0.13470088, 0.13470088, 0.13470088, 0.13470088, 0.13470088,
       0.13470088, 0.13470088, 0.13470088, 0.13470088, 0.13470088,
       0.13470088, 0.13470088, 0.13470088, 0.13470088, 0.13470088,
       0.13470088, 0.13470088, 0.13470088, 0.13470088, 0.13470088,
       0.13470088, 0.13470088, 0.13470088, 0.13470088, 0.13470088,
       0.13470088, 0.13470088, 0.13470088, 0.13470088, 0.13470088,
       0.13470088, 0.13470088, 0.13470088, 0.13470088, 0.13470088,
       0.13470088, 0.13470088, 0.13470088, 0.13470088, 0.13470088,
       0.13470088, 0.13470088, 0.13470088, 0.13470088, 0.13470088,
       0.12967784, 0.12326263, 0.12346414, 0.11431555, 0.12105048,
       0.12100501, 0.11642922, 0.12136666, 0.12150803, 0.12041785,
       0.12596364, 0.12831577, 0.13092311, 0.13255508, 0.13255

In [25]:
foreca(pure_sine)

0.9993976808641186

In [26]:
trailing_window(pure_sine, rolling_window=60, func=foreca)

array([0.87814292, 0.87814292, 0.87814292, 0.87814292, 0.87814292,
       0.87814292, 0.87814292, 0.87814292, 0.87814292, 0.87814292,
       0.87814292, 0.87814292, 0.87814292, 0.87814292, 0.87814292,
       0.87814292, 0.87814292, 0.87814292, 0.87814292, 0.87814292,
       0.87814292, 0.87814292, 0.87814292, 0.87814292, 0.87814292,
       0.87814292, 0.87814292, 0.87814292, 0.87814292, 0.87814292,
       0.87814292, 0.87814292, 0.87814292, 0.87814292, 0.87814292,
       0.87814292, 0.87814292, 0.87814292, 0.87814292, 0.87814292,
       0.87814292, 0.87814292, 0.87814292, 0.87814292, 0.87814292,
       0.87814292, 0.87814292, 0.87814292, 0.87814292, 0.87814292,
       0.87814292, 0.87814292, 0.87814292, 0.87814292, 0.87814292,
       0.87814292, 0.87814292, 0.87814292, 0.87814292, 0.87814292,
       0.85659439, 0.83599743, 0.81701635, 0.79995579, 0.78488091,
       0.77171463, 0.76030651, 0.75047711, 0.74204419, 0.73483677,
       0.72870169, 0.72350584, 0.7191359 , 0.71549702, 0.71251

# Hurst's Exponent

In [27]:
hurst_exponent(r)

(0.4448072860333123, 0.20713719369101494)

In [28]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    y=data["Close"]["PETR4.SA"],
    x=data.index,
    name="Preço PETR4.SA",
    yaxis="y1"
))


fig.show(fig_type="png")

In [29]:
np.random.seed(42)
n_points = 1000
trend = np.linspace(0, 10, n_points)
noise_trend = np.cumsum(standard_normal(n_points) * 0.1)
mean_reversion = np.zeros(n_points)
for t in range(1, n_points):
    mean_reversion[t] = mean_reversion[t-1] * 0.6 + standard_normal() * 0.5
brownian = np.cumsum(standard_normal(n_points))

In [30]:
hurst_exponent(noise_trend)

(0.6419241912818302, 0.000952023886777138)

In [31]:
hurst_exponent(mean_reversion)

(0.3685698852341455, 0.05402692976986088)

In [32]:
hurst_exponent(pure_sine)

(0.8575205638768405, 0.0014654668633357023)